**Initialise**

In [31]:
# This is a DreamerV2 implementation including replay memory and a target network 
# The model uses a RAM environment, converting bytes to 8 bit values. It shows very
# little tendency for exploration, preferring to shoot the alien that appears in the
# initial stage, rather than visiting planets.
# The code is loosely based on https://github.com/danijar/dreamerv2, which is released under the MIT licesne,
# however the implementation is largely done from scratch
# because the github implementation is written in tensorflow. It was, however, somewhat useful.


# imports
import gym
import collections
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as td

# hyperparameters
init_episodes     = 5
learning_rate     = 0.0005
gamma             = 0.995
lambda_hp         = 0.95
buffer_limit      = 50000
batch_size        = 50
video_every       = 5
print_every       = 1
horizon           = 15
latent_categories = 16
hidden_size       = 100

seq_len           = 50


class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n, l):
        start_points = [random.randint(0, len(self.buffer)) for _ in range(n)]
        mini_batch = random.sample(self.buffer, n)
        h_lst, xs_lst, as_lst, rs_lst, done_mask_lst = [], [], [], [], []
        
        for start in start_points:
            # Place start at start of the list
            self.buffer.rotate(-start)
            h, x, a, r, done_mask = self.buffer[0]
            h_lst.append(h)
            xs_lst.append(x.clone().detach())
            as_lst.append(a.clone().detach())
            rs_lst.append(r.clone().detach())
            done_mask_lst.append([done_mask])
            self.buffer.rotate(-1)
            for _, data in zip(range(l-1), self.buffer):
                _, x, a, r, done_mask = self.buffer[0]
                xs_lst[-1] = torch.cat((xs_lst[-1], x), dim=0)
                as_lst[-1] = torch.cat((as_lst[-1], a), dim=0)
                rs_lst[-1] = torch.cat((rs_lst[-1], r), dim=0)
                done_mask_lst[-1].append([done_mask])
            
            self.buffer.rotate(start+1)

        # return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
        #        torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
        #        torch.tensor(done_mask_lst)
        return h_lst, xs_lst, as_lst, rs_lst, done_mask_lst
    
    def size(self):
        return len(self.buffer)

class QNetwork(nn.Module):
    def __init__(self):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(np.array(env.observation_space.shape).prod(), 256)
        self.fc2 = nn.Linear(256, 84)
        self.fc3 = nn.Linear(84, env.action_space.n)

    def forward(self, x):
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(16, 17)
        else : 
            return out.argmax().item()

class WorldModel(nn.Module):
    def __init__(self, f=latent_categories):
        super(WorldModel, self).__init__()
        self.hidden_size = hidden_size
        self.f = f
        self.gru = nn.GRU(
            input_size=f*f + env.action_space.n,
            hidden_size=self.hidden_size
            )
        self.enc_layers = nn.Sequential(
            nn.Linear(1024 + self.hidden_size, 512),
            nn.LeakyReLU(),
            nn.Linear(512, f*f),
            nn.Unflatten(1, (f, f)),
            nn.Softmax(dim=2)   # log likelihood or probabilities?
        )
        self.dec_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(f*f + self.hidden_size, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 1024),
            nn.Sigmoid()
        )

        self.transition_pred = nn.Sequential(
            nn.Linear(self.hidden_size, 512),
            nn.LeakyReLU(),
            nn.Linear(512, f*f),
            nn.Unflatten(1, (f, f)),
            nn.Softmax(dim=2)   # log likelihood or probabilities?
        )

        self.reward_pred = nn.Sequential(
            nn.Linear(self.hidden_size + f*f, 50),
            nn.LeakyReLU(),
            nn.Linear(50, 1),
            nn.Sigmoid()
        )

    def ste(self, fn, x):
        dis = fn(probs=x)
        sample = dis.sample()
        # Add grad
        sample = sample + x - x.detach()
        return sample
    
    def encode(self, x, h):
        x = torch.cat((x, h), dim=1)
        x = self.enc_layers(x)
        return self.ste(td.OneHotCategorical, x)
    
    def decode(self, z, h):
        z = torch.cat((z, h), dim=1)
        return self.dec_layers(z)
    
    def forward(self, htm1, x_seq, a_seq):
        z_probs, z_hat_probs, x_hat_seq, r_hat_probs = [], [], [], []
        z_seq = []

        try:
            bs = htm1.size(1)
        except:
            bs = batch_size


        for x, a in zip(x_seq, a_seq):
            if htm1 is None:
                htm1 = torch.zeros(1, bs, self.hidden_size).to(device)
                h = htm1.view(bs, -1)
                # Encode a sequence of states
                z_prob = self.encode(x, h)
                z_hat_prob = z_prob.clone().detach()
                z = self.ste(td.OneHotCategorical, z_prob)
            else:
                h = htm1.view(bs, -1)
                z_hat_prob = self.transition_pred(h)
                z_prob = self.encode(x, h)
                z = self.ste(td.OneHotCategorical, z_prob)
            # print(z.shape)
            z = z.view(bs, self.f*self.f)
            # Decode latent representation
            x_hat = self.decode(z, h)
            r_hat = self.reward_pred(torch.cat((z, h), dim=1))

            rnn_input = torch.cat((z.view(1, bs, -1), a.view(1, bs, -1)), dim=2)
            _, htm1 = self.gru(rnn_input, htm1)
            
            z_seq.append(z)
            x_hat_seq.append(x_hat)
            r_hat_probs.append(r_hat)
            z_probs.append(z_prob)
            z_hat_probs.append(z_hat_prob)

        return htm1, z_seq, x_hat_seq, r_hat_probs, z_probs, z_hat_probs

class Actor(nn.Module):
    def __init__(self, f=latent_categories):
        super(Actor, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(f*f, 100),
            nn.LeakyReLU(),
            nn.Linear(100, env.action_space.n),
            nn.LogSoftmax(dim=-1)
        )
    
    def forward(self, x):
        return self.layers(x)

class Critic(nn.Module):
    def __init__(self, f=latent_categories):
        super(Critic, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(f*f, 100),
            nn.LeakyReLU(),
            nn.Linear(100, 1)
        )

    def forward(self, x):
        return self.layers(x)
            
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [32]:
def train_wm(model, memory, optimizer):
    model.zero_grad()

    h_lst, xs_lst, as_lst, rs_lst, done_mask_lst = memory.sample(batch_size, seq_len)
    h = torch.cat([h for h in h_lst], dim=1)
    x = torch.cat([x for x in xs_lst], dim=1)
    a = torch.cat([a for a in as_lst], dim=1)
    r = torch.cat([r for r in rs_lst], dim=1)
    
    # Run through model
    _, z_seq, x_hat_seq, r_hat_seq, z_probs, z_hat_probs = wm(h, x, a)

    try:
        print("initial loss:", total_loss)
    except:
        pass

    # Calculate how well model did
    x_hat = torch.cat([x.view(1, x.size(0), -1) for x in x_hat_seq], dim=0)
    recon_loss = bce(x_hat, x)
    r_hat = torch.cat([r.view(1, x.size(0)) for r in r_hat_seq], dim=0)
    reward_loss = mse(r_hat, r)

    z_hat = torch.cat([z.view(1, x.size(0), -1) for z in z_hat_probs], dim=0)
    z = torch.cat([z.view(1, x.size(0), -1) for z in z_seq], dim=0).detach()
    z_prob = torch.cat([z.view(1, x.size(0), -1) for z in z_probs], dim=0)
    trans_loss = 0.08 * bce(z_hat, z)
    ent_reg    = -0.02 * bce(z_prob, z)
    
    total_loss = recon_loss + reward_loss + trans_loss + ent_reg
    # print("total loss:", recon_loss, "+", reward_loss, "+", trans_loss, "+", ent_reg, "=", total_loss)

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

In [33]:
def dream(memory, wm, actor, critic, critic_target, optimizer_a, optimizer_c):
    h_lst, xs_lst, _, _, _ = memory.sample(batch_size, horizon)
    h = torch.cat([h for h in h_lst], dim=1).view(1, batch_size, -1)
    x = torch.cat([x[0].view(1, 1, -1) for x in xs_lst], dim=1).view(batch_size, -1)

    z = wm.encode(x, h.view(batch_size, -1)).detach()
    r_lst = []      # Actual rewards
    v_lst = []      # Estimate of long term value
    v_targets = []  # Likewise

    act_lls = []

    # Imagine up to horizon
    for i in range(horizon):
        # Take an action
        a = actor(z)
        act_lls.append(a)
        r_lst.append(wm.reward_pred(torch.cat((z.view(1, batch_size, -1), h), dim=2).view(batch_size, -1)).detach())

        # Feed into simulator
        rnn_input = torch.cat((z.view(1, batch_size, -1), torch.exp(a).view(1, batch_size, -1)), dim=2)
        _, h = wm.gru(rnn_input, h)
        z = wm.transition_pred(h.view(batch_size, -1)).detach()

        # Check rewards
        v_lst.append(critic(z))
        v_targets.append(critic_target(z).detach())
    
    # print(r_lst)
    V = [r_lst[-1] + gamma*v_targets[-1]]
    r_lst = reversed(r_lst[:-1])
    v_targets = reversed(v_targets[:-1])

    for r, v_target in zip(r_lst, v_targets):
        V.append(r + gamma*(1-lambda_hp)*v_target + lambda_hp*V[-1])
    
    critic_loss = sum([mse(v_est, v_tar) for v_est, v_tar in zip(v_lst, reversed(V))]) / horizon / 2.0
    optimizer_c.zero_grad()
    critic_loss.backward()
    optimizer_c.step()

    actor_loss = sum(sum(loss_t for loss_t in [-sum(act_ll*(V_t - v_t.detach())) for act_ll, v_t, V_t in zip(act_lls, v_lst, reversed(V))])) / env.action_space.n / batch_size / (horizon - 1)
    optimizer_a.zero_grad()
    actor_loss.backward()
    optimizer_a.step()
    
        

**Train**

← You can download the videos from the videos folder in the files on the left

In [36]:
# setup the Gravitar ram environment, and record a video every 50 episodes. You can use the non-ram version here if you prefer
env = gym.make('Gravitar-ram-v0')
env = gym.wrappers.Monitor(env, "./video", video_callable=lambda episode_id: (episode_id%video_every)==0,force=True)

# reproducible environment and action spaces, do not change lines 6-11 here (tools > settings > editor > show line numbers)
seed = 742
torch.manual_seed(seed)
env.seed(seed)
random.seed(seed)
np.random.seed(seed)
env.action_space.seed(seed)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
cpu = torch.device('cpu')

wm = WorldModel()
actor = Actor()
critic = Critic().to(device)
critic_target = Critic().to(device)
critic_target.load_state_dict(critic.state_dict())
memory = ReplayBuffer()

score    = 0.0
marking  = []
optimizer_wm = optim.Adam(wm.parameters(), lr=learning_rate)
optimizer_a = optim.Adam(actor.parameters(), lr=learning_rate)
optimizer_c = optim.Adam(critic.parameters(), lr=learning_rate)

bce = nn.BCELoss()
mse = nn.MSELoss()

for _ in range(init_episodes):
    done = False
    h = torch.zeros(1, 1, hidden_size)
    x = torch.from_numpy(np.unpackbits(env.reset())).float().view(1, 1, 1024)
    while not done:
        a = random.randint(0, env.action_space.n-1)
        x_prime, r, done, _ = env.step(a)
        a = F.one_hot(torch.LongTensor([[a]]), num_classes=env.action_space.n).float()
        memory.put((h.to(device), x.to(device), a.to(device), torch.tensor([[r/100.0]]).to(device), done))
        h, _, _, _, _, _ = wm(h, x, a)
        h = h.detach()
        x = torch.from_numpy(np.unpackbits(x_prime)).float().view(1, 1, 1024)
        
print(memory.size())

3578


In [35]:
env = gym.make('Gravitar-ram-v0')
env = gym.wrappers.Monitor(env, "./video", video_callable=lambda episode_id: (episode_id%video_every)==0,force=True)
for n_episode in range(int(1e32)):
    epsilon = max(0.01, 0.08 - 0.01*(n_episode/200)) # linear annealing from 8% to 1%
    wm = wm.to(device)
    actor = actor.to(device)
    for _ in range(20):
        # Dynamics learning
        train_wm(wm, memory, optimizer_wm)

        # Behaviour learning
        dream(memory, wm, actor, critic, critic_target, optimizer_a, optimizer_c)
    critic_target.load_state_dict(critic.state_dict())

    # Do a run
    wm = wm.to(cpu)
    actor = actor.to(cpu)
    print("brrr")
    x = torch.from_numpy(np.unpackbits(env.reset())).float().view(1, 1, 1024)
    h = torch.zeros(1, 1, hidden_size)
    done = False
    score = 0.0
    while not done:
        z = wm.encode(x.view(1, -1), h.view(1, -1))
        act_dis = td.Categorical(logits=actor(z))
        if random.random() < epsilon:
            a = random.randint(0, env.action_space.n-1)
        else:
            a = act_dis.sample()
        x_prime, r, done, info = env.step(a)
        a = F.one_hot(torch.LongTensor([[a]]), num_classes=env.action_space.n).float()
        memory.put((h.to(device), x.to(device), a.to(device), torch.tensor([[r/100.0]]).to(device), done))
        h, _, _, _, _, _ = wm(h, x, a)
        h = h.detach()
        x = torch.from_numpy(np.unpackbits(x_prime)).float().view(1, 1, 1024)

        score += r

    # do not change lines 44-48 here, they are for marking the submission log
    marking.append(score)
    if n_episode%100 == 0:
        print("marking, episode: {}, score: {:.1f}, mean_score: {:.2f}, std_score: {:.2f}".format(
            n_episode, score, np.array(marking).mean(), np.array(marking).std()))
        marking = []

        

    # you can change this part, and print any data you like (so long as it doesn't start with "marking")
    if n_episode%print_every==0 and n_episode!=0:
        print("episode: {}, score: {:.1f}, epsilon: {:.2f}".format(n_episode, score, epsilon))

brrr
marking, episode: 0, score: 250.0, mean_score: 250.00, std_score: 0.00
brrr
episode: 1, score: 200.0, epsilon: 0.08
brrr
episode: 2, score: 0.0, epsilon: 0.08
brrr
episode: 3, score: 350.0, epsilon: 0.08
brrr
episode: 4, score: 350.0, epsilon: 0.08
brrr
episode: 5, score: 0.0, epsilon: 0.08
brrr
episode: 6, score: 0.0, epsilon: 0.08
brrr
episode: 7, score: 0.0, epsilon: 0.08
brrr
episode: 8, score: 0.0, epsilon: 0.08
brrr
episode: 9, score: 0.0, epsilon: 0.08
brrr
episode: 10, score: 550.0, epsilon: 0.08
brrr
episode: 11, score: 500.0, epsilon: 0.08
brrr
episode: 12, score: 450.0, epsilon: 0.08
brrr
episode: 13, score: 250.0, epsilon: 0.08
brrr
episode: 14, score: 0.0, epsilon: 0.08
brrr
episode: 15, score: 450.0, epsilon: 0.08
brrr
episode: 16, score: 200.0, epsilon: 0.08
brrr
episode: 17, score: 500.0, epsilon: 0.08
brrr
episode: 18, score: 0.0, epsilon: 0.08
brrr
episode: 19, score: 0.0, epsilon: 0.08
brrr
episode: 20, score: 350.0, epsilon: 0.08
brrr
episode: 21, score: 100.0,

KeyboardInterrupt: ignored